In [1]:
library(dismo)
library(plyr)
library(rgbif)
library(GRaF)
library(pROC)
library(ROCR)
library(biogeo)
library(rgeospatialquality)
options(repr.plot.width = 20, repr.plot.height = 9)


Loading required package: raster
Loading required package: sp
Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following objects are masked from ‘package:stats’:

    cov, smooth, var

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



In [2]:
setwd("~/Dropbox/Projects/UW//SDMpriors")

In [3]:
data = read.csv("Sundayetal_thermallimits.csv")

In [4]:
data = subset(data, data$phylum=="Chordata")
head(data)

,species,tmax,tmin,tmax_metric,tmin_metric,lat,longitude,hemisphere,altitude,phylum,class,order,family,genus,sim_altitude,Ref.
71,Bufo_alvarius,39.9,2.5,crit,leth,31.00,-112.00,N,304.8,Chordata,Amphibia,Anura,Bufonidae,Bufo,553,Brattstrom_1968
72,Bufo_boreas,36.5,NA,crit,0,34.12,-116.82,N,2743.2,Chordata,Amphibia,Anura,Bufonidae,Bufo,2032,Brattstrom_1968
73,Bufo_canaliferus,37.3,9.5,crit,leth,14.58,-90.53,N,914.4,Chordata,Amphibia,Anura,Bufonidae,Bufo,1617,Brattstrom_1968
74,Bufo_canorus,38.0,-2.0,crit,leth,37.91,-119.26,N,2743.2,Chordata,Amphibia,Anura,Bufonidae,Bufo,3145,Brattstrom_1968
75,Bufo_cocifer,NA,2.8,0,leth,9.93,-84.08,N,1219.2,Chordata,Amphibia,Anura,Bufonidae,Bufo,1288,Brattstrom_1968
76,Bufo_cognatus,41.3,-4.0,crit,leth,31.00,-112.00,N,304.8,Chordata,Amphibia,Anura,Bufonidae,Bufo,553,Brattstrom_1968


In [5]:
data = data[!is.na(data$tmax) & !is.na(data$tmin),]
head(data)

,species,tmax,tmin,tmax_metric,tmin_metric,lat,longitude,hemisphere,altitude,phylum,class,order,family,genus,sim_altitude,Ref.
71,Bufo_alvarius,39.9,2.5,crit,leth,31.00,-112.00,N,304.8,Chordata,Amphibia,Anura,Bufonidae,Bufo,553,Brattstrom_1968
73,Bufo_canaliferus,37.3,9.5,crit,leth,14.58,-90.53,N,914.4,Chordata,Amphibia,Anura,Bufonidae,Bufo,1617,Brattstrom_1968
74,Bufo_canorus,38.0,-2.0,crit,leth,37.91,-119.26,N,2743.2,Chordata,Amphibia,Anura,Bufonidae,Bufo,3145,Brattstrom_1968
76,Bufo_cognatus,41.3,-4.0,crit,leth,31.00,-112.00,N,304.8,Chordata,Amphibia,Anura,Bufonidae,Bufo,553,Brattstrom_1968
77,Bufo_debilis,40.1,2.5,crit,leth,27.00,-112.00,N,304.8,Chordata,Amphibia,Anura,Bufonidae,Bufo,82,Brattstrom_1968
78,Bufo_exsul,37.1,-4.0,crit,leth,38.06,-84.45,N,1524.0,Chordata,Amphibia,Anura,Bufonidae,Bufo,291,Brattstrom_1968


In [6]:
data$spec = gsub("_"," ",data$species)
spec.id = 55

In [10]:
#key <- name_suggest(q=dat$spec[spec.id], rank='species')$key[1]

occ = occ_data(scientificName = data$spec[spec.id], limit=1000, minimal=FALSE)
occ = occ$data

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): Timeout was reached


In [11]:
gbifmap(occ, region="USA")

ERROR: Error in gbifmap(occ, region = "USA"): object 'occ' not found


In [ ]:
occ =  occ[which(!is.na(occ$"decimalLongitude") & !is.na(occ$"decimalLatitude"))  ,]


In [ ]:
length(occ)

In [ ]:
x = circles(occ[,c("decimalLongitude","decimalLatitude")], d=50000, lonlat=T)
bg = spsample(x@polygons, 100, type='random', iter=100)



In [ ]:
BClim = getData("worldclim", var="bio", res=2.5)
ext = extent(rbind(range(occ$decimalLongitude), range(occ$decimalLatitude))) # define the extent
BClim = crop(BClim, ext)


In [ ]:
occ_bc = extract(BClim, occ[,c("decimalLongitude","decimalLatitude")] ) # for the subsampled presence points
bg_bc = extract(BClim, bg) # for the pseudo-absence points
occ_bc = data.frame(lon=occ$decimalLongitude, lat=occ$decimalLatitude, occ_bc)
head(occ_bc)


In [ ]:
bgpoints = bg@coords
colnames(bgpoints) = c("lon","lat")
bg_bc = data.frame(cbind(bgpoints,bg_bc))
head(bg_bc)

In [ ]:
pres<-rep(1,dim(occ_bc)[1])
temp1 = data.frame(pres, occ_bc[,3:21])
temp1

In [ ]:
pres<-rep(1,dim(occ_bc)[1])
temp1<-data.frame(pres,occ_bc[,3:21])
pres<-rep(0,dim(bg_bc)[1])
temp2<-data.frame(pres,bg_bc[,3:21])
df<-rbind(temp1,temp2)
head(df,5)
